In [100]:
import sqlalchemy 
SQLALCHEMY_SILENCE_UBER_WARNING = 1
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy import create_engine, Table, Column, Integer, String, DateTime, ForeignKey, MetaData
from sqlalchemy.ext.declarative import declarative_base
from datetime import datetime

In [101]:
def connect_tcp_socket() -> sqlalchemy.engine.base.Engine:
    """ Initializes a TCP connection pool for a Cloud SQL instance of MySQL. """
    db_host = ''# e.g. '127.0.0.1' ('172.17.0.1' if deployed to GAE Flex)
    db_user = ''# e.g. 'my-db-user'
    db_pass = ''# e.g. 'my-db-password'
    db_name = ''# e.g. 'my-database'
    db_port =  # e.g. 3306

    engine = sqlalchemy.create_engine(
        sqlalchemy.engine.url.URL.create(
            drivername="mysql+pymysql",
            username=db_user,
            password=db_pass,
            host=db_host,
            port=db_port,
            database=db_name,
        ),
    )
    return engine

In [102]:
engine = connect_tcp_socket()
conn = engine.connect()
Base = declarative_base()

In [93]:
result = conn.execute("SHOW TABLES;").fetchall()
for r in result:
    print(r)

Conexion a la base de datos

In [74]:

result = conn.execute("SHOW COLUMNS FROM users;").fetchall()
for r in result:
    print(r)

('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('username', 'varchar(50)', 'NO', 'UNI', None, '')
('email', 'varchar(120)', 'NO', 'UNI', None, '')
('password', 'varchar(100)', 'NO', '', None, '')


In [82]:
result = conn.execute("SHOW COLUMNS FROM words;").fetchall()
for r in result:
    print(r)

('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('word', 'varchar(50)', 'YES', 'UNI', None, '')
('definition', 'varchar(500)', 'YES', '', None, '')
('user_id', 'int', 'YES', '', None, '')
('date', 'varchar(20)', 'YES', '', None, '')


In [103]:
engine.execute('DROP TABLE IF EXISTS users;')

In [104]:
Base.metadata.drop_all(bind=engine)

## **TABLA USERS**

In [105]:
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    username = Column(String(200), unique=True, nullable=False)
    email = Column(String(500), unique=True, nullable=False)
    password = Column(String(700), nullable=False)

# create table
Base.metadata.create_all(bind=engine)

## **TABLA WORDS**

In [107]:

metadata = MetaData()

words = Table('words', metadata,
    Column('id', Integer, primary_key=True),
    Column('word', String(300), unique=True),
    Column('definition', String(1000)),
    Column('user_id', Integer),
    Column('date', String(500))
)

metadata.create_all(engine)